In [1]:
%cd '/mnt/c/Users/yogi/Documents/Dev/cryptobot'



/mnt/c/Users/yogi/Documents/Dev/cryptobot


# Multedispatch solution 
The below uses a mutlidispatch solution. After building i figured it can be done with the built in singledispatch method. See Below

In [108]:
# For overloading symbol remap
from multipledispatch import dispatch

# For trading
from binance.client import Client
from binance.enums import *
import config

@dispatch(dict, str)
def mapper(mapping, pair):
    """ Converts symbols based on mapping dictionary
    """
    pair = pair.split('/')
    for i, sym in enumerate(pair):
        if sym in mapping:
#                 print('Changing {} to {}'.format(pair[i], self.remapping[sym]))
            pair[i] = mapping[sym]
    return ('/'.join(pair))

@dispatch(dict, list)
def mapper(mapping, pairs):
    return [mapper(mapping, x) for x in pairs]

@dispatch(dict, dict)
def mapper(mapping, pairs):
    return { mapper(mapping, s):t for s,t in pairs.items()}

class trader():
    def __init__(self):
        pass
    
    def remapDeco(func):
        def wrapper(self, pair):
            # Covert input from User to exchange symbols
            inp = mapper(self.remapping, pair)
            
            out = func(self,inp)
            
            # Convert output from exchange to user symbols
            out = mapper(self.unmapping, out)
            
            return out
        return wrapper
    
    def getPrice(self, sym): 
        """ Returns price for a single pair
        """
        pass

    def getAllPrices(self):
        """ Returns dictionary of all prices for a list of inputs
        """
        pass

class binance(trader):
    def __init__(self):
        self.client = Client(config.binance_api_key, config.binance_api_secret)
        self.remapping = {'BCH':'BCC','MIOTA':'IOTA', 'XRB':'NANO'}
        self.unmapping = { v:k for k,v in self.remapping.items()}
        self.prices = {'ADA/BTC': 43.0, 'BCC/BTC': 0.016, 'BTC': 0.002, 'BTG/BTC': 0.16, 'DASH/BTC': 0.03, 'EOS/BTC': 2.0, 'ETC/BTC': 0.83, 'ETH/BTC': 0.02, 'ICX/BTC': 3.42, 'IOTA/BTC': 9.0, 'LSK/BTC': 0.84, 'LTC/BTC': 0.11, 'NANO/BTC': 1.41, 'NEO/BTC': 0.14, 'QTUM/BTC': 0.68, 'TRX/BTC': 433.0, 'VEN/BTC': 4.0, 'XLM/BTC': 43.0, 'XMR/BTC': 0.07400000000000001, 'XRP/BTC': 21.0}

    @trader.remapDeco
    def getPrice(self, sym):
        return  {sym: (self.prices[sym], 'BTC')}
    
    @trader.remapDeco
    def getAllPrices(self, pairs): 
        prices = {}
        exchangePrices = self.client.get_all_tickers()
        # Scrub exchange prices
        exchangePrices = { x['symbol']:x['price'] for x in exchangePrices }

        for pair in pairs:
            
            prices[pair] = (exchangePrices[pair.replace("/","")], 'BTC')
        return prices
    
client = binance()

In [118]:
from functools import singledispatch

# For trading
from binance.client import Client
from binance.enums import *
import config

@singledispatch
def mapper(pair, mapping):
    raise NotImplementedError('Unsupported type')

@mapper.register(str)
def _(pair, mapping):
    """ Converts symbols based on mapping dictionary
    """
    pair = pair.split('/')
    for i, sym in enumerate(pair):
        if sym in mapping:
#                 print('Changing {} to {}'.format(pair[i], self.remapping[sym]))
            pair[i] = mapping[sym]
    return ('/'.join(pair))

@mapper.register(list)
def _(pairs, mapping):
    return [mapper(x, mapping) for x in pairs]

@mapper.register(dict)
def _(pairs, mapping):
    return { mapper(s, mapping):t for s,t in pairs.items()}

class trader():
    def __init__(self):
        pass
    
    def remapDeco(func):
        def wrapper(self, pair):
            # Covert input from User to exchange symbols
            inp = mapper(pair, self.remapping)
            
            out = func(self,inp)
            
            # Convert output from exchange to user symbols
            out = mapper(out, self.unmapping)
            
            return out
        return wrapper
    
    def getPrice(self, sym): 
        """ Returns price for a single pair
        """
        pass

    def getAllPrices(self):
        """ Returns dictionary of all prices for a list of inputs
        """
        pass

class binance(trader):
    def __init__(self):
        self.client = Client(config.binance_api_key, config.binance_api_secret)
        self.remapping = {'BCH':'BCC','MIOTA':'IOTA', 'XRB':'NANO'}
        self.unmapping = { v:k for k,v in self.remapping.items()}
        self.prices = {'ADA/BTC': 43.0, 'BCC/BTC': 0.016, 'BTC': 0.002, 'BTG/BTC': 0.16, 'DASH/BTC': 0.03, 'EOS/BTC': 2.0, 'ETC/BTC': 0.83, 'ETH/BTC': 0.02, 'ICX/BTC': 3.42, 'IOTA/BTC': 9.0, 'LSK/BTC': 0.84, 'LTC/BTC': 0.11, 'NANO/BTC': 1.41, 'NEO/BTC': 0.14, 'QTUM/BTC': 0.68, 'TRX/BTC': 433.0, 'VEN/BTC': 4.0, 'XLM/BTC': 43.0, 'XMR/BTC': 0.07400000000000001, 'XRP/BTC': 21.0}

    @trader.remapDeco
    def getPrice(self, sym):
        return  {sym: (self.prices[sym], 'BTC')}
    
    @trader.remapDeco
    def getAllPrices(self, pairs): 
        prices = {}
        exchangePrices = self.client.get_all_tickers()
        # Scrub exchange prices
        exchangePrices = { x['symbol']:x['price'] for x in exchangePrices }

        for pair in pairs:
            
            prices[pair] = (exchangePrices[pair.replace("/","")], 'BTC')
        return prices
    
client = binance()

In [146]:
from functools import singledispatch

# For trading
from binance.client import Client
from binance.enums import *
import config


class exchange():
    def __init__(self):
        self.mapping = {}
        self.unmapping = {}
        pass

    
    @singledispatch
    @classmethod    
    def mapper(pair, mapping):
        raise NotImplementedError('Unsupported type')

    @mapper.register(str)
    def _(pair, mapping):
        """ Converts symbols based on mapping dictionary
        """
        pair = pair.split('/')
        for i, sym in enumerate(pair):
            if sym in mapping:
    #                 print('Changing {} to {}'.format(pair[i], self.remapping[sym]))
                pair[i] = mapping[sym]
        return ('/'.join(pair))

    @mapper.register(list)
    def _(pairs, mapping):
        return [ trader.mapper(x, mapping) for x in pairs ]

    @mapper.register(dict)
    def _(pairs, mapping):
        return { trader.mapper(s, mapping):t for s,t in pairs.items()}
    
    def remapDeco(func):
        def wrapper(self, pair):
            # Covert input from User to exchange symbols
            inp = trader.mapper(pair, self.mapping)
            
            out = func(self,inp)
            
            # Convert output from exchange to user symbols
            out = trader.mapper(out, self.unmapping)
            
            return out
        return wrapper
    
    def getPrice(self, pair): 
        """ Returns price for a single pair
        """
        pass

    def getAllPrices(self, pairs):
        """ Returns dictionary of all prices for a list of inputs
        """
        pass

class binance(exchange):
    def __init__(self):
        self.client = Client(config.binance_api_key, config.binance_api_secret)
        self.mapping = {'BCH':'BCC','MIOTA':'IOTA', 'XRB':'NANO'}
        self.unmapping = { v:k for k,v in self.mapping.items()}
        self.prices = {'ADA/BTC': 43.0, 'BCC/BTC': 0.016, 'BTC': 0.002, 'BTG/BTC': 0.16, 'DASH/BTC': 0.03, 'EOS/BTC': 2.0, 'ETC/BTC': 0.83, 'ETH/BTC': 0.02, 'ICX/BTC': 3.42, 'IOTA/BTC': 9.0, 'LSK/BTC': 0.84, 'LTC/BTC': 0.11, 'NANO/BTC': 1.41, 'NEO/BTC': 0.14, 'QTUM/BTC': 0.68, 'TRX/BTC': 433.0, 'VEN/BTC': 4.0, 'XLM/BTC': 43.0, 'XMR/BTC': 0.07400000000000001, 'XRP/BTC': 21.0}

    @trader.remapDeco
    def getPrice(self, pair):
        return  {sym: (self.prices[pair], 'Price')}
    
    @trader.remapDeco
    def getAllPrices(self, pairs): 
        prices = {}
        exchangePrices = self.client.get_all_tickers()
        # Scrub exchange prices
        exchangePrices = { x['symbol']:x['price'] for x in exchangePrices }

        for pair in pairs:
            
            prices[pair] = (exchangePrices[pair.replace("/","")], 'Price')
        return prices
    
client = binance()

In [147]:
print(client.getAllPrices(['BCH/BTC','MIOTA/BTC','DASH/BTC']))

{'BCH/BTC': ('0.12219400', 'Price'), 'MIOTA/BTC': ('0.00019934', 'Price'), 'DASH/BTC': ('0.06101700', 'Price')}


In [141]:
print(client.getAllPrices(['BCH/BTC','MIOTA/BTC','DASH/BTC']))

{'BCH/BTC': ('0.12204100', 'Price'), 'MIOTA/BTC': ('0.00019899', 'Price'), 'DASH/BTC': ('0.06103900', 'Price')}


In [142]:
print(client.getPrice('MIOTA/BTC'))

# print(client.symbolRemapping('BCH'))

{'MIOTA/BTC': (9.0, 'Price')}


In [78]:
client.prices

{'ADA/BTC': 43.0,
 'BCC/BTC': 0.016,
 'BTC': 0.002,
 'BTG/BTC': 0.16,
 'DASH/BTC': 0.03,
 'EOS/BTC': 2.0,
 'ETC/BTC': 0.83,
 'ETH/BTC': 0.02,
 'ICX/BTC': 3.42,
 'IOTA/BTC': 9.0,
 'LSK/BTC': 0.84,
 'LTC/BTC': 0.11,
 'NANO/BTC': 1.41,
 'NEO/BTC': 0.14,
 'QTUM/BTC': 0.68,
 'TRX/BTC': 433.0,
 'VEN/BTC': 4.0,
 'XLM/BTC': 43.0,
 'XMR/BTC': 0.07400000000000001,
 'XRP/BTC': 21.0}

In [ ]:
def symbolRemap(func):
    def wrapper(self, x):
        remap = {'BCH':'BCC'}
        x = remap[x]
        print("Changed to binance pair mapping:")
        func(self,x)
    return wrapper

@symbolRemap
def price(x):
    print(x)

price("BCH")

In [22]:
pair = 'ADA/BTC'.split('/')
'/'.join(pair)

'ADA/BTC'

In [20]:
for i in [quote, base]:
    print(i)

ADA
BTC
